<a href="https://colab.research.google.com/github/nermal1/Stock-Market-Prediction/blob/main/alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [24]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.requests import MarketOrderRequest

import pandas as pd
import datetime
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense

In [25]:
API_KEY = "PKYI39LXLT3K2SSXGBC7"
API_SECRET = "yzpMAtY5CCLxq2azyBgony0fzZeYgznPqMfzU0sU"

data_client = StockHistoricalDataClient(API_KEY, API_SECRET)
trading_client = TradingClient(API_KEY, API_SECRET, paper=True)

In [26]:
start = datetime.datetime(2020, 6, 1)
end = datetime.datetime(2025, 7, 1)
symbols = ["JPM", "KULR", "META", "MS", "MU", "NVDA", "OKLO", "AVGO"]

In [17]:
account = trading_client.get_account()
available_cash = float(account.cash)

positions = trading_client.get_all_positions()
held_symbols = {pos.symbol for pos in positions}


In [27]:
request_params = StockBarsRequest(
    symbol_or_symbols=symbols,
    timeframe=TimeFrame.Day,
    start=start,
    end=end
)
bars = data_client.get_stock_bars(request_params).df

In [28]:
symbol_dfs = {}
for symbol in symbols:
    symbol_df = bars.loc[bars.index.get_level_values('symbol') == symbol, ['close']]
    symbol_df.rename(columns={'close': 'Close'}, inplace=True)
    symbol_dfs[symbol] = symbol_df

In [29]:
def preprocess_data(df, window_size=60):
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)

    X, y = [], []
    for i in range(window_size, len(scaled)):
        X.append(scaled[i - window_size:i, 0])
        y.append(scaled[i, 0])

    X = np.array(X).reshape(-1, window_size, 1)
    y = np.array(y)
    return X, y, scaler

In [30]:
def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [43]:
import os

models_dir = "trained_models"
os.makedirs(models_dir, exist_ok=True)

# Train and save model per stock
for symbol in symbols:
    df = symbol_dfs[symbol]
    if len(df) < 100:
        print(f"Skipping {symbol}, not enough data.")
        continue

    X, y, scaler = preprocess_data(df)

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    print(f"Training model for {symbol}")
    model.fit(X, y, epochs=20, batch_size=32, verbose=0)

    # Save model
    model.save(f"{models_dir}/{symbol}_lstm_model.h5")
    print(f"Saved model for {symbol}")

    # Save scaler too (optional but helpful for consistent predictions)
    import joblib
    joblib.dump(scaler, f"{models_dir}/{symbol}_scaler.save")



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training model for JPM


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Saved model for JPM
Training model for KULR


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Saved model for NVDA
Training model for OKLO


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Saved model for OKLO
Training model for AVGO


Saved model for AVGO


In [44]:
def predict_next_price(model, df, scaler, window_size=60):
    X, _, _ = preprocess_data(df, window_size)
    last_input = X[-1].reshape(1, window_size, 1)
    pred_scaled = model.predict(last_input)
    predicted_price = scaler.inverse_transform([[pred_scaled[0][0]]])[0][0]
    last_price = df['Close'].iloc[-1]
    return predicted_price, last_price

In [45]:
def generate_signal(predicted, current, threshold=0.01):
    change = (predicted - current) / current
    if change > threshold:
        return "buy"
    elif change < -threshold:
        return "sell"
    return "hold"

In [46]:
def execute_trade(symbol, signal, current_price, held_symbols, available_cash):
    if signal == "buy":
        if symbol in held_symbols:
            print(f"Already holding {symbol}, skipping buy.")
            return
        if available_cash < current_price:
            print(f"Not enough cash to buy {symbol}. Needed: ${current_price:.2f}, Available: ${available_cash:.2f}")
            return
        order = MarketOrderRequest(
            symbol=symbol,
            qty=1,
            side=OrderSide.BUY,
            time_in_force=TimeInForce.DAY
        )
        trading_client.submit_order(order)
        print(f"BUY 1 share of {symbol} at ~${current_price:.2f}")

    elif signal == "sell":
        if symbol not in held_symbols:
            print(f"No holdings to sell in {symbol}, skipping.")
            return
        order = MarketOrderRequest(
            symbol=symbol,
            qty=1,
            side=OrderSide.SELL,
            time_in_force=TimeInForce.DAY
        )
        trading_client.submit_order(order)
        print(f"SELL 1 share of {symbol} at ~${current_price:.2f}")

    else:
        print(f"HOLD {symbol}")

In [47]:
account = trading_client.get_account()
available_cash = float(account.cash)
positions = trading_client.get_all_positions()
held_symbols = {pos.symbol for pos in positions}

window_size = 60

for symbol in symbols:
    df = symbol_dfs[symbol]
    if len(df) < window_size + 1:
        print(f"Not enough data for {symbol}")
        continue

    try:
        model = load_model(f"trained_models/{symbol}_lstm_model.h5")
        scaler = joblib.load(f"trained_models/{symbol}_scaler.save")
    except:
        print(f"No model or scaler found for {symbol}, skipping.")
        continue

    predicted, current = predict_next_price(model, df, scaler)
    signal = generate_signal(predicted, current)
    execute_trade(symbol, signal, current, held_symbols, available_cash)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 478ms/step
No holdings to sell in JPM, skipping.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step
No holdings to sell in KULR, skipping.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step


No holdings to sell in META, skipping.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step


No holdings to sell in MS, skipping.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step


BUY 1 share of MU at ~$123.25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step


Already holding NVDA, skipping buy.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step


No holdings to sell in OKLO, skipping.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
SELL 1 share of AVGO at ~$275.65
